# Sprint 4 Round-Up
Работа с Jupyter Notebook.<br>
Для запуска и работы в notebook нужно установить зависимости: <br>
* `pip install -r requirements.txt`
* И запустить Jupyter Notebook `python -m manage shell_plus --notebook`



In [1]:
# Imports
import os
from books.models import Book, Store, Publisher, StoreChain
from django.db.models import Prefetch
from books.utils import load_items
from books.decorators import query_debugger
import datetime
import logging

# Settings for Notebook
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Reset DB and create fresh DB with some data for books amd publishers
load_items()

#Enabling logging queries for future tasks if needed

log = logging.getLogger('django.db.backends')
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())

## CRUD
•	Create: Model.objects.create() — создание объекта в базе

•	Read: Model.objects.get(id=N) — чтение объекта по его ключу

•	Update: object.property= 'new value' и потом object.save() — изменение объекта

•	Delete: object.delete() — удаление объекта из базы

In [2]:
#Examples
publisher_hp = Publisher.objects.get(name="Publisher1")
book_new = Book.objects.create(name="Harry Potter",price=350,issued=datetime.date(1995, 9, 12), publisher=publisher_hp)
#Новая книга уже в БД

book_new.name = "Harry Potter and Philosopher Stone"
book_new.save()

print(Book.objects.get(id=book_new.id))
book_new.delete()
try:
    print(Book.objects.get(id=book_new.id))
except Exception as e:
    print('Такой книги уже нет')


(0.000) SELECT "books_publisher"."id", "books_publisher"."name", "books_publisher"."country" FROM "books_publisher" WHERE "books_publisher"."name" = 'Publisher1' LIMIT 21; args=('Publisher1',)
(0.016) INSERT INTO "books_book" ("name", "price", "issued", "publisher_id") VALUES ('Harry Potter', 350, '1995-09-12', 401); args=['Harry Potter', 350, '1995-09-12', 401]
(0.006) UPDATE "books_book" SET "name" = 'Harry Potter and Philosopher Stone', "price" = 350, "issued" = '1995-09-12', "publisher_id" = 401 WHERE "books_book"."id" = 90007; args=('Harry Potter and Philosopher Stone', 350, '1995-09-12', 401, 90007)
(0.000) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" WHERE "books_book"."id" = 90007 LIMIT 21; args=(90007,)
(0.000) BEGIN; args=None
(0.000) DELETE FROM "books_store_books" WHERE "books_store_books"."book_id" IN (90007); args=(90007,)
(0.000) DELETE FROM "books_book" WHERE "books_book"."id" I

Harry Potter and Philosopher Stone
Такой книги уже нет


## Фильтрация объектов

filter

fieldname__operator

Example,
text__contains='text'

text__exact='Exact phrase'

### Operators:

1. exact
2. iexact
3. contains
4. in
5. gt — > (больше),
6. gte — => (больше или равно),
7. lt — < (меньше),
8. lte — <= (меньше или равно).
9. Операторы сравнения с началом и концом строки startswith, endswith
10. range — вхождение в диапазон
11. isnull — проверка на пустое значение.


https://docs.djangoproject.com/en/3.1/topics/db/queries/

In [4]:
# Example

# Простой фильтр
publisher5 = Publisher.objects.get(name='Publisher5')
published_by_fifth = Book.objects.filter(publisher=publisher5)
print(len(published_by_fifth)) #Полный запрос к БД

(0.000) SELECT "books_publisher"."id", "books_publisher"."name", "books_publisher"."country" FROM "books_publisher" WHERE "books_publisher"."name" = 'Publisher5' LIMIT 21; args=('Publisher5',)
(0.000) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" WHERE "books_book"."publisher_id" = 405; args=(405,)


200


In [5]:
# Фильтр с оператором
# Поиск изданий между датами
start_search = datetime.date(1800,1,1)
end_search = datetime.date(1850,1,1)
pub_bet_dates = Book.objects.filter(issued__range=(start_search, end_search))
print(len(pub_bet_dates))

(0.000) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" WHERE "books_book"."issued" BETWEEN '1800-01-01' AND '1850-01-01'; args=('1800-01-01', '1850-01-01')


2321


In [6]:
# Фильтр по полю связанного объекта
books_from_norway_publisher = Book.objects.filter(publisher__country='Norway')
print(len(books_from_norway_publisher))

(0.000) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_publisher" ON ("books_book"."publisher_id" = "books_publisher"."id") WHERE "books_publisher"."country" = 'Norway'; args=('Norway',)


1200


In [7]:
# Фильтр
books_from_u = Book.objects.filter(publisher__country__startswith='U')
print(len(books_from_u))


# То же самое другим способом
publishers_u = Publisher.objects.filter(country__startswith='U')
print(len(Book.objects.filter(publisher__in=publishers_u)))

(0.001) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_publisher" ON ("books_book"."publisher_id" = "books_publisher"."id") WHERE "books_publisher"."country" LIKE 'U%' ESCAPE '\'; args=('U%',)
(0.000) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" WHERE "books_book"."publisher_id" IN (SELECT U0."id" FROM "books_publisher" U0 WHERE U0."country" LIKE 'U%' ESCAPE '\'); args=('U%',)


1800
1800


In [8]:
# Фильтр, объединение условий
start_search = datetime.date(1800,1,1)
end_search = datetime.date(1810,1,1)
published_by_uk = Book.objects.filter(issued__range=(start_search, end_search)).filter(publisher__country="UK")
print(published_by_uk)

(0.001) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_publisher" ON ("books_book"."publisher_id" = "books_publisher"."id") WHERE ("books_book"."issued" BETWEEN '1800-01-01' AND '1810-01-01' AND "books_publisher"."country" = 'UK') LIMIT 21; args=('1800-01-01', '1810-01-01', 'UK')


<QuerySet [<Book: Book404>, <Book: Book409>, <Book: Book429>, <Book: Book434>, <Book: Book482>, <Book: Book486>, <Book: Book502>, <Book: Book536>, <Book: Book6242>, <Book: Book6263>, <Book: Book6282>, <Book: Book6299>, <Book: Book6304>, <Book: Book6317>, <Book: Book6323>, <Book: Book6329>, <Book: Book6397>, <Book: Book6640>, <Book: Book6665>, <Book: Book6693>, '...(remaining elements truncated)...']>


In [9]:
# Ограничение и сортировка
published_by_uk.order_by("-name")[:10]

(0.000) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_publisher" ON ("books_book"."publisher_id" = "books_publisher"."id") WHERE ("books_book"."issued" BETWEEN '1800-01-01' AND '1810-01-01' AND "books_publisher"."country" = 'UK') ORDER BY "books_book"."name" DESC LIMIT 10; args=('1800-01-01', '1810-01-01', 'UK')


<QuerySet [<Book: Book7770>, <Book: Book7769>, <Book: Book7753>, <Book: Book7749>, <Book: Book7746>, <Book: Book7742>, <Book: Book7708>, <Book: Book7702>, <Book: Book7683>, <Book: Book7621>]>

In [10]:
# Ошибка! Сначала должна быть сортировка
try:
    published_by_uk[:10].order_by("-name")
except Exception as e:
    print(e)

Cannot reorder a query once a slice has been taken.


In [11]:
# Tasks
# Найти количество книг дороже 150 с издателем в России и их сумма (два запроса)

## Загрузка связанных записей

https://docs.djangoproject.com/en/3.1/ref/models/querysets/

<b>select_related</b>

Returns a QuerySet that will “follow” foreign-key relationships, selecting additional related-object data when it executes its query. This is a performance booster which results in a single more complex query but means later use of foreign-key relationships won’t require database queries.

<b>prefetch_related</b>

Returns a QuerySet that will automatically retrieve, in a single batch, related objects for each of the specified lookups.

This has a similar purpose to select_related, in that both are designed to stop the deluge of database queries that is caused by accessing related objects, but the strategy is quite different.

select_related works by creating an SQL join and including the fields of the related object in the SELECT statement. For this reason, select_related gets the related objects in the same database query. However, to avoid the much larger result set that would result from joining across a ‘many’ relationship, select_related is limited to single-valued relationships - foreign key and one-to-one.

In [12]:
@query_debugger
def naive_books():
    queryset = Book.objects.all()
    books = []
    for book in queryset:
        books.append({'id': book.id, 'name': book.name, 'publisher': book.publisher.name})
    
@query_debugger
def run_prefetch_books():
    queryset = Book.objects.prefetch_related('publisher').all()
    books = []
    for book in queryset:
        books.append({'id': book.id, 'name': book.name, 'publisher': book.publisher.name})

@query_debugger
def run_select_books():
    queryset = Book.objects.select_related('publisher').all()
    books = []
    for book in queryset:
        books.append({'id': book.id, 'name': book.name, 'publisher': book.publisher.name})

@query_debugger
def run_prefetch_store():
    queryset = Store.objects.prefetch_related('books').all()
    stores = []
    for store in queryset:
        stores.append({'id': store.id, 'name': store.name, 'books': store.books})

In [13]:
# naive_books()

In [14]:
run_prefetch_books()

(0.002) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book"; args=()
(0.000) SELECT "books_publisher"."id", "books_publisher"."name", "books_publisher"."country" FROM "books_publisher" WHERE "books_publisher"."id" IN (401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450); args=(401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450)


Function : run_prefetch_books
Number of Queries : 2
Finished in : 0.19s


In [15]:
run_select_books()

(0.001) SELECT "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id", "books_publisher"."id", "books_publisher"."name", "books_publisher"."country" FROM "books_book" INNER JOIN "books_publisher" ON ("books_book"."publisher_id" = "books_publisher"."id"); args=()


Function : run_select_books
Number of Queries : 1
Finished in : 0.26s


In [16]:
run_prefetch_store()

(0.001) SELECT "books_store"."id", "books_store"."name", "books_store"."marginal_price" FROM "books_store"; args=()
(0.000) SELECT ("books_store_books"."store_id") AS "_prefetch_related_val_store_id", "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_store_books" ON ("books_book"."id" = "books_store_books"."book_id") WHERE "books_store_books"."store_id" IN (801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900); args=(801, 802, 803, 804, 805, 806, 807,

Function : run_prefetch_store
Number of Queries : 2
Finished in : 0.16s


In [17]:
# select_related не работает для many-to-many полей
try:
    len(Store.objects.select_related('books').all())
except Exception as e:
    print(e)

Invalid field name(s) given in select_related: 'books'. Choices are: (none)


In [18]:
# Multiple examples

In [19]:
list(StoreChain.objects.prefetch_related(Prefetch('stores__books')))

(0.000) SELECT "books_storechain"."id", "books_storechain"."name", "books_storechain"."best_store_id" FROM "books_storechain"; args=()
(0.000) SELECT ("books_storechain_stores"."storechain_id") AS "_prefetch_related_val_storechain_id", "books_store"."id", "books_store"."name", "books_store"."marginal_price" FROM "books_store" INNER JOIN "books_storechain_stores" ON ("books_store"."id" = "books_storechain_stores"."store_id") WHERE "books_storechain_stores"."storechain_id" IN (35, 36, 37, 38, 39); args=(35, 36, 37, 38, 39)
(0.000) SELECT ("books_store_books"."store_id") AS "_prefetch_related_val_store_id", "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_store_books" ON ("books_book"."id" = "books_store_books"."book_id") WHERE "books_store_books"."store_id" IN (801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 

[<StoreChain: Chain1>,
 <StoreChain: Chain2>,
 <StoreChain: Chain3>,
 <StoreChain: Chain4>,
 <StoreChain: Chain5>]

In [20]:
list(StoreChain.objects.prefetch_related(Prefetch('best_store__books')))

(0.000) SELECT "books_storechain"."id", "books_storechain"."name", "books_storechain"."best_store_id" FROM "books_storechain"; args=()
(0.000) SELECT "books_store"."id", "books_store"."name", "books_store"."marginal_price" FROM "books_store" WHERE "books_store"."id" IN (899, 845, 821, 891, 892); args=(899, 845, 821, 891, 892)
(0.000) SELECT ("books_store_books"."store_id") AS "_prefetch_related_val_store_id", "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_store_books" ON ("books_book"."id" = "books_store_books"."book_id") WHERE "books_store_books"."store_id" IN (821, 845, 891, 892, 899); args=(821, 845, 891, 892, 899)


[<StoreChain: Chain1>,
 <StoreChain: Chain2>,
 <StoreChain: Chain3>,
 <StoreChain: Chain4>,
 <StoreChain: Chain5>]

In [21]:
# 3 Queries
chains = list(StoreChain.objects.prefetch_related('stores__books'))

(0.002) SELECT "books_storechain"."id", "books_storechain"."name", "books_storechain"."best_store_id" FROM "books_storechain"; args=()
(0.000) SELECT ("books_storechain_stores"."storechain_id") AS "_prefetch_related_val_storechain_id", "books_store"."id", "books_store"."name", "books_store"."marginal_price" FROM "books_store" INNER JOIN "books_storechain_stores" ON ("books_store"."id" = "books_storechain_stores"."store_id") WHERE "books_storechain_stores"."storechain_id" IN (35, 36, 37, 38, 39); args=(35, 36, 37, 38, 39)
(0.000) SELECT ("books_store_books"."store_id") AS "_prefetch_related_val_store_id", "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_store_books" ON ("books_book"."id" = "books_store_books"."book_id") WHERE "books_store_books"."store_id" IN (801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 

In [76]:
# 3 Queries
StoreChain.objects.prefetch_related(Prefetch('best_store__books'))

(0.000) SELECT "books_storechain"."id", "books_storechain"."name", "books_storechain"."best_store_id" FROM "books_storechain" LIMIT 21; args=()
(0.000) SELECT "books_store"."id", "books_store"."name", "books_store"."marginal_price" FROM "books_store" WHERE "books_store"."id" IN (899, 845, 821, 891, 892); args=(899, 845, 821, 891, 892)
(0.000) SELECT ("books_store_books"."store_id") AS "_prefetch_related_val_store_id", "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_store_books" ON ("books_book"."id" = "books_store_books"."book_id") WHERE "books_store_books"."store_id" IN (821, 845, 891, 892, 899); args=(821, 845, 891, 892, 899)


<QuerySet [<StoreChain: Chain1>, <StoreChain: Chain2>, <StoreChain: Chain3>, <StoreChain: Chain4>, <StoreChain: Chain5>]>

In [23]:
# 2 Queries
StoreChain.objects.select_related('best_store').prefetch_related('best_store__books')

(0.000) SELECT "books_storechain"."id", "books_storechain"."name", "books_storechain"."best_store_id", "books_store"."id", "books_store"."name", "books_store"."marginal_price" FROM "books_storechain" INNER JOIN "books_store" ON ("books_storechain"."best_store_id" = "books_store"."id") LIMIT 21; args=()
(0.000) SELECT ("books_store_books"."store_id") AS "_prefetch_related_val_store_id", "books_book"."id", "books_book"."name", "books_book"."price", "books_book"."issued", "books_book"."publisher_id" FROM "books_book" INNER JOIN "books_store_books" ON ("books_book"."id" = "books_store_books"."book_id") WHERE "books_store_books"."store_id" IN (821, 845, 892, 891, 899); args=(821, 845, 892, 891, 899)


<QuerySet [<StoreChain: Chain1>, <StoreChain: Chain2>, <StoreChain: Chain3>, <StoreChain: Chain4>, <StoreChain: Chain5>]>

## Агрегирующие записи
Avg: вернёт среднее значение по указанной колонке в выборке

Count: вернёт количество записей в выборке, как и метод count(), описанный выше

Max: вернёт максимальное значение по указанной колонке в выборке

Min: вернёт минимальное значение по указанной колонке в выборке

Sum: вернёт сумму значений по указанной колонке в выборке

In [24]:
Store.objects.filter(chains__name='Chain3').count()

(0.000) SELECT COUNT(*) AS "__count" FROM "books_store" INNER JOIN "books_storechain_stores" ON ("books_store"."id" = "books_storechain_stores"."store_id") INNER JOIN "books_storechain" ON ("books_storechain_stores"."storechain_id" = "books_storechain"."id") WHERE "books_storechain"."name" = 'Chain3'; args=('Chain3',)


19

In [25]:
Book.objects.filter(store__name='Store2').aggregate(Avg('price'))

(0.004) SELECT AVG("books_book"."price") AS "price__avg" FROM "books_book" INNER JOIN "books_store_books" ON ("books_book"."id" = "books_store_books"."book_id") INNER JOIN "books_store" ON ("books_store_books"."store_id" = "books_store"."id") WHERE "books_store"."name" = 'Store2'; args=('Store2',)


{'price__avg': 160.34}

In [26]:
Book.objects.exclude(store__chains__name__in=['Chain1','Chain2']).filter(price__gt=150).aggregate(Sum('price'))

(0.008) SELECT SUM("books_book"."price") AS "price__sum" FROM "books_book" WHERE (NOT ("books_book"."id" IN (SELECT U1."book_id" FROM "books_store_books" U1 INNER JOIN "books_store" U2 ON (U1."store_id" = U2."id") INNER JOIN "books_storechain_stores" U3 ON (U2."id" = U3."store_id") INNER JOIN "books_storechain" U4 ON (U3."storechain_id" = U4."id") WHERE U4."name" IN ('Chain1', 'Chain2'))) AND "books_book"."price" > 150); args=('Chain1', 'Chain2', 150)


{'price__sum': 833632}

In [27]:
# Связь между таблицами
# Найдем лучший магазин в 1 сетке
Store.objects.filter(championed_by__name='Chain1')

(0.003) SELECT "books_store"."id", "books_store"."name", "books_store"."marginal_price" FROM "books_store" INNER JOIN "books_storechain" ON ("books_store"."id" = "books_storechain"."best_store_id") WHERE "books_storechain"."name" = 'Chain1' LIMIT 21; args=('Chain1',)


<QuerySet [<Store: Store21>]>

In [28]:
# Книг в магазине
annotated_results = Store.objects.annotate(books_count = Count('books'))

In [29]:
print([item.books_count for item in list(annotated_results)])

(0.000) SELECT "books_store"."id", "books_store"."name", "books_store"."marginal_price", COUNT("books_store_books"."book_id") AS "books_count" FROM "books_store" LEFT OUTER JOIN "books_store_books" ON ("books_store"."id" = "books_store_books"."store_id") GROUP BY "books_store"."id", "books_store"."name", "books_store"."marginal_price"; args=()


[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


In [30]:
# Задание: посчитать количество магазинов в сетке

## Closures and decorators

### Local functions

LEGB Rule (Local, enclosing, global, built-in)

In [31]:
g = 'global'
def outer(p='param'):
        l = 'local'
        def inner():
            print(g, p, l)
            
        inner()
outer()

global param local


### Returning functions

Functions are treated as any other object

In [32]:
def enclosing():
    def local_func():
        print('local func')
    return local_func
        
lf = enclosing()
lf()

local func


### Closures and nested scopes

Closure — maintain a reference to objects from earlier scopes

In [33]:
def enclosing():
    x = 'closed over'
    def local_func():
        print(x)
    return local_func
        
lf = enclosing()
lf()
lf.__closure__

closed over


(<cell at 0x10d255b10: str object at 0x10d255170>,)

### Function factories

function that returns new specialized functions

In [34]:
def raise_to(exp):
    def raise_to_exp(x):
        return pow(x, exp)
    return raise_to_exp

square = raise_to(2)
print(square.__closure__)
print(square(5))
cube = raise_to(3)
print(cube(5))

(<cell at 0x10d2596d0: int object at 0x1074c5b50>,)
25
125


### Decorators

Modify or enhance functions without changing their definitions

Takes a callable and returns a callable

In [35]:
def animal():
    return 'bjørn'

print(animal())

def animal():
    return ascii('bjørn')
print(animal())

bjørn
'bj\xf8rn'


In [36]:
def escape_unicode(f):
    def wrap(*args, **kwargs): #accepting any number of arguments
        x = f(*args, **kwargs) #calling the f with those arguments
        return ascii(x)
    return wrap

In [37]:
@escape_unicode
def northern_city():
    return 'Tromsø'

print(northern_city())

'Troms\xf8'


In [38]:
# Order of decorators = reversed

def prepend_index(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return  kwargs['index'] + x
    return wrap

def prepend_district(f):
    def wrapping(*var, **dicts):
        x = f(*var, **dicts)
        return var[0] + x
    return wrapping

In [39]:
@prepend_index
@prepend_district
@escape_unicode
def southern_city(district, **kwargs):
    return 'Øslo'

print(southern_city('Tråx', index='123øL'))
    

123øLTråx'\xd8slo'


### Extended Formal Argument Syntax

In [40]:
# Почему это работает?
print()
print("one")
print("one","two")
print("one","two", "three")


one
one two
one two three


In [41]:
# *args -> для работы с positional arguments
# Расчитаем объем n-мерного куба

def hypervolume(*args):
    print(args)
    print(type(args))
    volume = 1
    for length in args:
        volume*=length
    return volume

In [42]:
print(hypervolume(10))
print(hypervolume(10, 10))
print(hypervolume(10, 10, 10))
print(hypervolume(2, 4, 6, 8))

(10,)
<class 'tuple'>
10
(10, 10)
<class 'tuple'>
100
(10, 10, 10)
<class 'tuple'>
1000
(2, 4, 6, 8)
<class 'tuple'>
384


In [43]:
hypervolume() # incorrect -> should raise an error

()
<class 'tuple'>


1

In [44]:
def hypervolume(length, *lengths):
    volume = length
    for length in lengths:
        volume*=length
    return volume

In [45]:
print(hypervolume(10, 4))
try:
    print(hypervolume()) # raises an error
except Exception as e:
    print(e)

40
hypervolume() missing 1 required positional argument: 'length'


In [46]:
# extended syntax
# def extended(*args, **kwargs):

In [47]:
def tag(name, **kwargs):
    print(name)
    print(kwargs)
    print(type(kwargs))

In [48]:
tag('img', src='monet.jpg', alt="Sunrise by lake", border=1)

img
{'src': 'monet.jpg', 'alt': 'Sunrise by lake', 'border': 1}
<class 'dict'>


In [49]:
def tag(name, **attributes):
    result = '<' + name
    for key, value in attributes.items():
        result += f' {key}="{value}"'
    result += '>'
    return result

In [50]:
tag('img', src='monet.jpg', alt="Sunrise by lake", border=1)

'<img src="monet.jpg" alt="Sunrise by lake" border="1">'

In [51]:
# Неправильные синтаксисы

In [52]:
# def print_args(**kwargs, *args):
#     pass

In [53]:
def print_args(arg1, arg2, *args):
    print(arg1)
    print(arg2)
    print(args)

print_args(1, 2, 3, 4, 5, 6)

1
2
(3, 4, 5, 6)


In [54]:
def print_args(arg1, arg2, *args, kwarg1, kwarg2):
    print(arg1)
    print(arg2)
    print(args)
    print(kwarg1)
    print(kwarg2)

In [55]:
print_args(1, 2, 3, 4, 5, 6, kwarg1='k1', kwarg2='k2')

1
2
(3, 4, 5, 6)
k1
k2


In [57]:
print_args(1, 2, 3, 4, 5, 6, kwarg1='k1')   

TypeError: print_args() missing 1 required keyword-only argument: 'kwarg2'

In [58]:
def print_args(arg1, arg2, *args, kwarg1, kwarg2, **kwargs):
    print(arg1)
    print(arg2)
    print(args)
    print(kwarg1)
    print(kwarg2)
    print(kwargs)

In [59]:
print_args(1, 2, 3, 4, 5, 6, kwarg1='k1', kwarg2='k2')

1
2
(3, 4, 5, 6)
k1
k2
{}


In [60]:
print_args(1, 2, 3, 4, 5, 6, kwarg1='k1', kwarg2='k2', kwarg3="k3")

1
2
(3, 4, 5, 6)
k1
k2
{'kwarg3': 'k3'}


In [61]:
# Extended actual argument syntax
def print_args(arg1, arg2, *args):
    print(arg1)
    print(arg2)
    print(args)

print_args(1,2,3,4)

t = (1, 2, 3, 4)
print_args(*t)

1
2
(3, 4)
1
2
(3, 4)


In [62]:
def color(red, green, blue, **kwargs):
    print("r =", red)
    print("g =", green)
    print("b =", blue)
    print(kwargs)
    
k = {'red':21, 'green': 45, 'blue': 98, 'alpha':100}
color(**k)

r = 21
g = 45
b = 98
{'alpha': 100}


In [63]:
## Forwarding arguments
def trace(f, *args, **kwargs):
    print("args =", args)
    print("kwargs =", kwargs)
    result = f(*args, )
    print("result = ", result)
    return result

In [64]:
int('1001', base=2)

9

In [65]:
trace(int, '1001', base=2)

args = ('1001',)
kwargs = {'base': 2}
result =  1001


1001

In [66]:
arr = [[1, 2, 3, 4],
       [5, 6, 7, 8],
       [9, 10, 11, 12]]

In [67]:
help(zip)

Help on class zip in module builtins:

class zip(object)
 |  zip(iter1 [,iter2 [...]]) --> zip object
 |  
 |  Return a zip object whose .__next__() method returns a tuple where
 |  the i-th element comes from the i-th iterable argument.  The .__next__()
 |  method continues until the shortest iterable in the argument sequence
 |  is exhausted and then it raises StopIteration.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [68]:
for item in zip(arr[0], arr[1], arr[2]):
    print(item)

(1, 5, 9)
(2, 6, 10)
(3, 7, 11)
(4, 8, 12)


In [69]:
transposed = list(zip(*arr))

In [70]:
print(transposed)

[(1, 5, 9), (2, 6, 10), (3, 7, 11), (4, 8, 12)]
